# Notebook do jednowymiarowej symulacji FDTD

Importowanie bibliotek

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

import numpy as np

Ustalamy stałe: $\epsilon_0$ i $\mu_0$. W celu weryfikacji obliczamy prędkość światła $c$.

In [ ]:
epsilon0 = 625000/(22468879468420441*np.pi) 
mu0 = np.pi/2500000
c = 1/np.sqrt(epsilon0*mu0)
c

Definiiujemy parametry siatki: 
- długość przewodu $l$
- krok siatki $dx$
- krok czasowy $dt$

Na ich podstawie ustawiamy wielkość siatki: $nx$


In [ ]:
l = 500 # m
dx = 1  #m
dt = 10e-10 # s
nx = int(l/dx)
nx

Sprawdzamy warunek zbieżności

In [ ]:
dx/dt > c

Alokacja siatki na $E_z$ oraz $H_y$, a także wartości $\sigma$

In [ ]:
ez = np.zeros(nx)
hy = np.zeros(nx)

sigma = 0.00001 * np.ones(nx)

L = 50
for i in range(L):
    sigma[i] = 1
    sigma[nx-i-1] = 1


#L = 50
#for i in range(L):
#    sigma[i] *= np.exp((L-i)/10)
#    sigma[nx-i-1] *= np.exp((L-i)/10)


In [ ]:
x = np.linspace(0, 1, nx)
plt.plot(x,sigma)
plt.show()

Usjalamy źródło w postaci sinusoidy jednostkowej

In [ ]:
p = int(nx/2)

def source(time):
    p = int(nx/2)
    return p, np.sin(2 * np.pi * (0 - 0.005 * time))

Zmienne pomocnicze usprawniające obliczenia (stałe w czasie współczynniki obliczamy raz, a nie za każdym razem)

In [ ]:
coef_e_loss = [ (epsilon0 - sigma[i]*dt/2)/(epsilon0 + sigma[i]*dt/2) for i in range(nx) ]
coef_e_step = [ dt/(dx*(epsilon0 + sigma[i]*dt/2)) for i in range(nx) ]
coef_h_step = [ dt/(dx*mu0) for i in range(nx) ]
coef_j = [ dt/(epsilon0 + sigma[i]*dt/2) for i in range(nx) ]

In [ ]:
coef_e_loss[3]

In [ ]:
coef_e_step[3]

In [ ]:
coef_h_step[3]

Funkcje pomocnicze wykonujące kroki symulacji. Warunki brzegowe ustalamy jako zerowe pole.

In [ ]:
#@jit(nopython=True)
def e_interior_update():
    for i in range(1,nx-1):
        ez[i] = coef_e_loss[i]*ez[i] + coef_e_step[i]*(hy[i]-hy[i-1])
        
#@jit(nopython=True)
def h_interior_update():
    for i in range(1,nx-1):
        hy[i] = hy[i] + coef_h_step[i]*(ez[i+1]-ez[i])
        
#@jit(nopython=True)
#def e_current_apply(ez, coeff, j):
#    for jt in j:
#        ez[jt[0]] = ez[jt[0]] - coeff[jt[0]] * jt[1]
        
def e_boundary_conditions():
    #ez[0] = ez[-1]
    pass
    
def h_boundary_conditions():
    #hy[0] = hy[-1]
    pass

def e_source(time):
    point, val = source(t)
    if point is not None:
        ez[point] = val
        ez[point-1] = 0.8*val
        ez[point+1] = 0.8*val
            

In [ ]:
# create figure and axes 
fig, (ax1, ax2) = plt.subplots(2, figsize=(8, 4))
ax1.plot([0.9, 0.9], [-2, 2], 'k-')
ax1.plot([0.1, 0.1], [-2, 2], 'k-')
ax2.plot([0.9, 0.9], [-2, 2], 'k-')
ax2.plot([0.1, 0.1], [-2, 2], 'k-')

# creating our line objects for the plots
ezplot, = ax1.plot(x, ez, '-b')
hyplot, = ax2.plot(x, hy, '-r')

def animate(t):
    """
    funkcja wywoływana przez animacje w celu obliczenia kolejnego kroku symulacji
    
    t: czas
    
    return:
        ezplot: ydata
        hyplot: ydata
    """
    
    for i in range(1):
        e_interior_update()
        #e_boundary_conditions()
        ez[p]= source(t)[1]

        h_interior_update()
        #h_boundary_conditions()
    
    ezplot.set_ydata(ez)
    hyplot.set_ydata(hy)


def init():
    """
    initialize the figure
    """
    
    ax1.set_ylim(-1, 1)
    ax2.set_ylim(-0.005, 0.005)

    ax1.set_xlim(0, 1)
    ax2.set_xlim(0, 1)
    ax1.axhline(0, color='black', lw=1)
    ax2.axhline(0, color='black', lw=1)
    plt.rcParams.update({'font.size':14})
    
    return ezplot, hyplot,

ani = FuncAnimation(fig, animate, init_func=init, interval=50, blit=True)
plt.show()